In [1]:
import numpy as np
import os
import json
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load Final Model (YOLOv5 from Experiment 3)

In [6]:
%cd /content/drive/My Drive
# !pip install ultralytics
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp2/weights/best.pt', force_reload=True)

/content/drive/My Drive


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-4-6 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7045186 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


## Heuristics

In [7]:
warm_to_hot_clothing = ['skirt', 'shorts', 'short sleeve top', 'sling dress','vest',
                        'vest dress', 'short sleeve dress', 'sling']
warm_clothing = ['short sleeve outwear'] # everything in warm_to_hot plus this
cool_clothing = ['trousers', 'long sleeve dress', 'long sleeve top', 'long sleeve outwear']

In [8]:
def heuristics_w_temp_range(detected_categories, temperature):
    # warm to hot
    if temperature <= 30 and temperature >= 25:
        print("Weather is warm to hot:")
        if isinstance(detected_categories, list): # max two items
            for c in detected_categories:
                if c in warm_to_hot_clothing:
                    print(f"{c} is suitable for this weather!")
                else:
                    print(f"{c} may not be suitable. Consider changing.")
        else:
            if c in warm_to_hot_clothing:
                print(f"{c} is suitable for this weather!")
            else:
                print(f"{c} may not be suitable. Consider changing.")
    elif temperature < 25 and temperature >= 20:
        print("Weather is warm:")
        if isinstance(detected_categories, list): # max two items
            for c in detected_categories:
                if c in warm_to_hot_clothing or c in warm_clothing:
                    print(f"{c} is suitable for this weather!")
                else:
                    print(f"{c} may not be suitable. Consider changing.")
        else:
            if c in warm_to_hot_clothing or c in warm_clothing:
                print(f"{c} is suitable for this weather!")
            else:
                print(f"{c} may not be suitable. Consider changing.")
    elif temperature < 20 and temperature >= 15:
        print("Weather is cool:")
        if isinstance(detected_categories, list): # max two items
            for c in detected_categories:
                if c in cool_clothing:
                    print(f"{c} is suitable for this weather!")
                else:
                    print(f"{c} may not be suitable. Consider changing.")
        else:
            if c in cool_clothing:
                print(f"{c} is suitable for this weather!")
            else:
                print(f"{c} may not be suitable. Consider changing.")
    else:
        print("Invalid temperature range! Enter temperature between 15 to 30 degrees celsius")


## Integration of Model and Heuristics

In [66]:
def is_outfit_appropriate(image, current_weather):
  prediction = model(image)
  prediction_str = str(prediction)
  start_index = prediction_str.index('352x352') + len('352x352')
  end_index = prediction_str.index('Speed')
  extracted_string = prediction_str[start_index:end_index].strip()

  predictions_list = extracted_string.split(',')
  # Process each prediction to extract the class name
  classes = [prediction.split(' ')[-2:] for prediction in predictions_list]

  # Join the class names into a list
  for i in range(len(classes)):
    if classes[i][0].isnumeric():
      classes[i][0] = ' '
  class_names = [' '.join(class_name).strip() for class_name in classes]

  heuristics_w_temp_range(class_names, current_weather)


## Test System

In [68]:
image_ids = ['im2652', 'im26469', 'im5736']
temps = [22, 10, 27]
test_images_dir = '/content/drive/My Drive/YOLOv5_work/images/test'

for i in image_ids:
  for j in temps:
    image = os.path.join(test_images_dir,  i + '.jpg')
    print(f'For {i}, and temperature {j} degrees celsius:')
    is_outfit_appropriate(image, j)
    print()


For im2652, and temperature 22 degrees celsius:
Weather is warm:
sleeve outwear may not be suitable. Consider changing.

For im2652, and temperature 10 degrees celsius:
Invalid temperature range! Enter temperature between 15 to 30 degrees celsius

For im2652, and temperature 27 degrees celsius:
Weather is warm to hot:
sleeve outwear may not be suitable. Consider changing.

For im26469, and temperature 22 degrees celsius:
Weather is warm:
vest is suitable for this weather!

For im26469, and temperature 10 degrees celsius:
Invalid temperature range! Enter temperature between 15 to 30 degrees celsius

For im26469, and temperature 27 degrees celsius:
Weather is warm to hot:
vest is suitable for this weather!

For im5736, and temperature 22 degrees celsius:
Weather is warm:
skirt is suitable for this weather!
vest dress is suitable for this weather!

For im5736, and temperature 10 degrees celsius:
Invalid temperature range! Enter temperature between 15 to 30 degrees celsius

For im5736, and